In [54]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"
dataset_name = "LOD"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"
input_cancer_class = "pan_cancer"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
os.system(f"mkdir -p {path_to_08_output}")
all_read_classification_files = [item for item in pathlib.Path(path_to_07_output).glob("*.raw.read_classification.csv")]
all_candi_read_files = [item for item in pathlib.Path(path_to_07_output).glob("*candi_reads.csv")]

metadata = pd.read_csv(f"./metadata/{dataset_name}.csv")
metadata = metadata[metadata["Sample"].duplicated() == False]

if os.path.isfile(os.path.join(path_to_08_output, "feature.csv")) == False:
    fulldf = pd.DataFrame()
    for i in tqdm(range(len(all_candi_read_files))):
        tmp_readdf = pd.read_csv(all_read_classification_files[i], index_col = [0])
        tmp_candidf = pd.read_csv(all_candi_read_files[i], index_col = [0])
        sampleid = all_candi_read_files[i].name.replace(".candi_reads.csv", "")
        raw_count = tmp_readdf.shape[0]
        in_read_count = tmp_readdf[tmp_readdf["read_overlap_rate"] == "in"].shape[0]
        mean_candi_reads = tmp_candidf.candi_reads.mean()
        ratio_raw = mean_candi_reads/raw_count
        ratio_in_read = mean_candi_reads/in_read_count
        tmpdf = pd.DataFrame({"SampleID": sampleid, 
                            "raw_count": raw_count, 
                            "in_read_count": in_read_count, 
                            "mean_candi_reads": mean_candi_reads,
                            "ratio_raw": ratio_raw,
                            "ratio_in_read": ratio_in_read}, index = [0])

        fulldf = pd.concat([fulldf, tmpdf], axis = 0)
    fulldf.to_csv(os.path.join(path_to_08_output, "feature.csv"))
else:
    fulldf = pd.read_csv(os.path.join(path_to_08_output, "feature.csv"), index_col = [0])


100%|██████████| 427/427 [00:59<00:00,  7.21it/s]


In [57]:
fulldf

,SampleID,raw_count,in_read_count,mean_candi_reads,ratio_raw,ratio_in_read
0,LODLIV12,18031,9085,4.684366,0.000260,0.000516
0,LODBREAST18R2,19666,10266,4.967262,0.000253,0.000484
0,LODCONTROL3,12360,6358,2.975535,0.000241,0.000468
0,LIVER3R9,21510,11133,10.637168,0.000495,0.000955
0,LODLIV15R1,47104,24513,19.255193,0.000409,0.000786
...,...,...,...,...,...,...
0,LODCRC18R2,14400,7232,3.187500,0.000221,0.000441
0,CONTROL31R2,9162,4580,2.192926,0.000239,0.000479
0,LODLUNG19R2,26695,13462,6.557864,0.000246,0.000487
0,GASTRIC2,19746,10215,5.183976,0.000263,0.000507


In [47]:
metadata[metadata["Sample"].isin(fulldf["sampleid"].unique())]

,Sample,Actual tumor_fraction_ichorCNA,Simulated TF,spike-in,LABEL,LABCODE,Detected,Decision,RUNTM,RUNGW
0,LODCRC6,0.02443,0.003226,1,Colorectal cancer,CRC8,No,Healthy,R3596,R3606
1,BREAST10R1,0.02187,0.02187,100,Breast cancer,BREAST10,No,Healthy,R3725,R3724
2,LODCRC3PR1,0.07539,0.056745,15,Colorectal cancer,CRC8,No,Healthy,R3549,R3549
3,LODBREAST7R2,0.02290,0.0001145,0.5,Breast cancer,BREAST7,No,Healthy,R3596,R3606
4,LODBREAST6,0.02238,0.0006958,1,Breast cancer,BREAST7,No,Healthy,R3596,R3606
...,...,...,...,...,...,...,...,...,...,...
674,CONTROL44R9,0.03381,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3715,R3715
675,LODCONTROL2,0.02951,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3683,R3683
676,LODCONTROL3,0.02966,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3703,R3703
677,LODCONTROL4R1,0.04060,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3715,R3715
